In [1]:
from db_handlers import load_from_database

# Example for SQL Server (Windows Auth)
df1 = load_from_database(
)

# Example for PostgreSQL
df2 = load_from_database(
)

TypeError: load_sql_server_data() missing 1 required positional argument: 'args'

In [4]:
from db_handlers import load_from_database
df2 = load_from_database()
print(df2.head())
    

TypeError: load_sql_server_data() missing 1 required positional argument: 'args'

In [9]:
from ayx import Alteryx as ax
import pandas as pd
import numpy as np


def calculate_daily_null_and_other_percentages(df):
    results = []
    grouped = df.groupby("Date")

    for date, group in grouped:
        total_rows = len(group)

        for column in group.columns:
            if column == "Date":
                continue
            null_count = group[column].apply(lambda x: pd.isna(x) or str(x).strip() == "").sum()
                

            # "Other" detection
            other_count = group[column].apply(
                lambda x: (str(x).strip().lower() == "other" or str(x).strip().lower=="others") if pd.notnull(x) else False).sum()

            # Percentages
            null_percentage = (null_count / total_rows) 
            other_percentage = (other_count / total_rows) 

            results.append({
                "Date": date,
                "Dimension": column,
                "Null Count": null_count,
                "Other Count": other_count,
                "Null Percentage (%)": round(null_percentage, 2),
                "Other Percentage (%)": round(other_percentage, 2),
            })

    return pd.DataFrame(results)

df_fr=ax.read("#1")
df_fr=df_fr[~(df_fr['IssueDate'].isna())]
df_fr['Date']=[str(i)[:10] for i in df_fr['IssueDate']]
df_sales=ax.read("#2")
df_sales=df_sales[~(df_sales['BeneficiaryCreationDate'].isna())]
df_sales['Date']=[str(i)[:10] for i in df_sales['BeneficiaryCreationDate']]
df_titles=ax.read("#3")
df_titles=df_titles[~(df_titles['CreationTime'].isna())]
df_titles['Date']=[str(i)[:10] for i in df_titles['CreationTime']]

results_df_fr = calculate_daily_null_and_other_percentages(df_fr)
results_df_sales = calculate_daily_null_and_other_percentages(df_sales)
results_df_titles = calculate_daily_null_and_other_percentages(df_titles)

ax.write(results_df_fr,1)
ax.write(results_df_sales,2)
ax.write(results_df_titles,3)

In [3]:
import pandas as pd
from data_quality_checker import DataQualityChecker  # adjust if using inline/not module
import webbrowser
import os
df2

,CreatedAt,AreaSize,SellingPriceAmount,PropertyUse,PropertyType,unittype,ParcelNumber,PlanNumber,DistrictNameAr,CityNameAr,...,SpatialUnitSouthernBorderLength,SpatialUnitEasternBorderLength,SpatialUnitWesternBorderLength,realestate_number,CaseNumber,DeedSerial,DeedNumber,moj_deed_date,new_type,new_use
0,2025-06-03 08:27:48.2874528,400.20,950000.0,Residential,Residential Building,None,1023/1,2705/أ,الدار البيضاء,الرياض,...,29.993,13.3341,13.3339,4444320901800000,TR202527427,1731170,710205003929,22/10/1433,فيلا,Residential
1,2025-04-07 16:03:21.8020590,147.64,775000.0,Mixed Use,Apartment,Apartment,23,3171,الروابي,الرياض,...,32,49.9916,50.0001,4445698939200008,TR202518199,13203179,398523000312,9/6/1443,شقة,Residential
2,2025-02-06 08:03:13.0395337,876.24,1.0,Residential,Empty Land,None,588,ش د 845,الفنار,الدمام,...,30.1,29.34,29.26,4512080061100000,TR20257111,14626635,430127005434,1444/04/28,أرض,Residential
3,2025-01-22 14:44:12.2265183,312.00,592800.0,Residential,Empty Land,None,4134,3968,النظيم,الرياض,...,12,26,26,4530775211300000,TR20254132,16368282,218511004021,19/8/1445,أرض,Residential
4,2024-07-02 10:56:25.4184921,529.89,9300000.0,Residential,Residential Building,None,81/2,3776,الملقا,الرياض,...,19.7986,44.6288,36.4541,4557395423200000,TR20243718,13281567,617807004146,24/6/1443,فيلا,Residential
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42346,2024-07-29 08:37:03.5133037,186.80,1850000.0,Mixed Use,Residential Building,DoubleFloorApartment,154/1,1822/ج,قرطبة,الرياض,...,24.9396,30.0259,30.0566,9549680978600004,DT20240576,None,None,None,شقة,Residential
42347,2025-03-10 18:13:48.2316727,428.32,360000.0,Residential,Empty Land,None,1727,3097,عريض,الرياض,...,17.51,25,25,3840655467500000,TR202513956,9896196,399001000480,15/12/1441,أرض,Residential
42348,2024-11-18 07:26:56.4386715,290.42,1740000.0,Residential,Floor,Floor,180/1,2078/ج,العارض,الرياض,...,45.0945,19.9999,20,5042962798600001,TR202417632,None,None,None,دور,Residential
42349,2024-11-10 10:59:58.3188977,289.92,980000.0,Residential,Residential Building,None,666/2,3314/11,بدر,الرياض,...,11.9667,24.1601,24.2262,6649879767400000,TR202416207,10091669,310122046270,20/1/1442,فيلا,Residential


In [5]:
checker = DataQualityChecker(df2)
checker.run_all_checks()
checker.print_summary()

# Export to HTML
output_path = "data_quality_report.html"
checker.export_html_report(output_path)

# Open report in browser
webbrowser.open('file://' + os.path.abspath(output_path))



===== Null Values =====
                                 Missing Count  Missing %
CreatedAt                                    0       0.00
AreaSize                                     0       0.00
SellingPriceAmount                          34       0.08
PropertyUse                                  0       0.00
PropertyType                                 7       0.02
unittype                                 27587      65.14
ParcelNumber                                 0       0.00
PlanNumber                                   1       0.00
DistrictNameAr                               0       0.00
CityNameAr                                   0       0.00
RegionNameAr                                 0       0.00
SpatialUnitNorthernBorderLength              8       0.02
SpatialUnitSouthernBorderLength              8       0.02
SpatialUnitEasternBorderLength               8       0.02
SpatialUnitWesternBorderLength               8       0.02
realestate_number                            0 

True

In [27]:
import os
import pandas as pd
from pathlib import Path
import pandas as pd
import logging

# Logger setup
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)
if not logger.handlers:
    handler = logging.StreamHandler()
    formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
    handler.setFormatter(formatter)
    logger.addHandler(handler)

# Import file handlers
from file_handlers.load_csv import load_csv
from file_handlers.load_excel import load_excel
from file_handlers.load_json import load_json
from file_handlers.load_parquet import load_parquet
from file_handlers.load_txt import load_txt
from file_handlers.load_xml import load_xml
from file_handlers.load_compressed import load_compressed
from db_handlers import load_from_database
# Import DB & Web handlers
from db_handlers.sql_loader import load_sql_server_data
from db_handlers.postgres_loader import load_postgres_data
#from web_handlers.api_loader import load_api
#from web_handlers.html_scraper import load_html

# Utility
#from utils.logger import logger
#from utils.file_detector import detect_file_type


def load_data(source: str, source_type: str = None, **kwargs) -> pd.DataFrame:
    """
    Main function to load data from any source.
    
    Parameters:
    - source: str | URL, file path, SQL query, or API endpoint
    - source_type: str | Explicit type like 'csv', 'sql', 'api' (optional)
    - kwargs: dict | Extra arguments passed to specific loaders
    
    Returns:
    - DataFrame
    """
    logger.info(f"Loading data from source: {source}")

    # Auto-detect if type is not passed
    if not source_type:
        source_type = detect_file_type(source)
        logger.info(f"Auto-detected source type: {source_type}")
   
    loaders = {
        'csv': load_csv,
        'excel': load_excel,
        'json': load_json,
        'parquet': load_parquet,
        'txt': load_txt,
        'xml': load_xml,
        'zip': load_compressed,
        'sql': load_sql_server_data,
        'postgres': load_postgres_data
    }
    
    loader_func = loaders.get(source_type)
    if not loader_func:
        raise ValueEr
        ror(f"Unsupported source_type: {source_type}")
    return loader_func(source, **kwargs)

df_parquet=load_data('sample.parquet', 'parquet')
df_json=load_data('mappings.json', 'json')
df_excel=load_data('2024 data Riyadh.xlsx', 'excel',sheet_name='Qitah')
df_zip=load_data('messy_data.zip','zip')
df_csv=load_data('messy_data.csv','csv')



2025-06-16 13:36:31,500 - INFO - Loading data from source: sample.parquet
2025-06-16 13:36:31,505 - INFO - Loading data from source: mappings.json
2025-06-16 13:36:31,513 - INFO - Loading data from source: 2024 data Riyadh.xlsx
2025-06-16 13:36:31,597 - INFO - Loading data from source: messy_data.zip
2025-06-16 13:36:31,603 - INFO - Loading data from source: messy_data.csv


In [29]:
df_parquet

,id,name,age
0,1,Alice,30
1,2,Bob,25


In [31]:
df_json

,nationality_map1,country_map2,nationality_map3
السويدية,سويدي,NaN,NaN
أسباني,أسباني,NaN,دول الإتحاد الأوربي
بولندي,بولندي,NaN,دول الإتحاد الأوربي
جيبوتي,جيبوتي,NaN,دول الجامعة العربية الأخرى
سوري,سوري,NaN,دول الجامعة العربية الأخرى
...,...,...,...
توفالو,NaN,NaN,دول الأوقيانوسية
فانواتو,NaN,NaN,دول الأوقيانوسية
واليس وفوتونا,NaN,NaN,دول الأوقيانوسية
القارة القطبية الجنوبية,NaN,NaN,دول أخرى


In [35]:
df_csv

,Name,Age,Salary,Department,JoiningDate
0,Dana,50.0,80000.0,Sales,2023-03-10
1,Eve,45.0,NaN,NaN,2019-06-15
2,Charlie,50.0,80000.0,NaN,2021-07-30
3,Eve,50.0,30000.0,NaN,NaN
4,Eve,NaN,999999.0,NaN,2019-06-15
...,...,...,...,...,...
1195,Alice,150.0,30000.0,IT,NaN
1196,Alice,50.0,70000.0,NaN,2021-07-30
1197,Dana,45.0,45000.0,IT,NaN
1198,Eve,25.0,70000.0,NaN,2020-01-01


In [8]:
import requests
import pandas as pd
from utils.logger import logger

def load_api(url: str, headers: dict = None, params: dict = None, response_format: str = 'json', **kwargs) -> pd.DataFrame:
    """
    Load data from an API endpoint returning JSON or CSV.
    
    Parameters:
    - url: API endpoint URL
    - headers: optional HTTP headers
    - params: query parameters for GET
    - response_format: 'json' or 'csv'
    - kwargs: extra params to pd.DataFrame or pd.read_csv
    
    Returns:
    - pd.DataFrame
    """
    logger.info(f"Loading data from API: {url}")
    try:
        response = requests.get(url, headers=headers, params=params)
        response.raise_for_status()

        if response_format.lower() == 'json':
            data = response.json()
            # If the JSON is a list of records, load directly
            if isinstance(data, list):
                return pd.DataFrame(data, **kwargs)
            # If JSON is nested, user should preprocess or specify a path
            elif isinstance(data, dict):
                # Try to find common key for records or let user specify via kwargs
                # This is a simple heuristic; can be extended
                for key, val in data.items():
                    if isinstance(val, list):
                        return pd.DataFrame(val, **kwargs)
                # fallback: flatten the dict into a DataFrame
                return pd.json_normalize(data, **kwargs)
            else:
                raise ValueError("Unsupported JSON format from API")
        elif response_format.lower() == 'csv':
            from io import StringIO
            csv_data = StringIO(response.text)
            return pd.read_csv(csv_data, **kwargs)
        else:
            raise ValueError("Unsupported response_format. Use 'json' or 'csv'")
    except Exception as e:
        logger.error(f"API loading failed: {e}")
        raise


In [10]:


import pandas as pd
from utils.logger import logger

def load_html(url: str, match: str = None, **kwargs) -> pd.DataFrame:
    """
    Load tables from an HTML page URL.

    Parameters:
    - url: str | Webpage URL with tables
    - match: str | String to match table(s) by text in the table (optional)
    - kwargs: extra params passed to pd.read_html

    Returns:
    - pd.DataFrame | If multiple tables, returns the first one by default
    """
    logger.info(f"Loading HTML tables from {url}")

    try:
        tables = pd.read_html(url, match=match, **kwargs)
        if not tables:
            raise ValueError("No tables found on the web page")
        return tables[0]  # Return first table by default
    except Exception as e:
        logger.error(f"HTML loading failed: {e}")
        raise


In [14]:
from sqlalchemy import create_engine
import pandas as pd
import getpass
from urllib.parse import quote_plus
import urllib

def load_sql_server_data(port='1433'):
    print(port)
    host = input("SQL Server host/IP: ").strip()
    port=input('Portnumber(default is 1433):').strip()
    database = input("Database name: ").strip()
    table = input("Table name: ").strip()
    query=input("write/paste your query").strip()
    if port=='':
        port='1433'

    # Proper formatting with comma before port
    server = f"{host},{port}"
    print(server)

    conn_str = (
    "Driver={ODBC Driver 17 for SQL Server};"
    f"Server={host};"
    f"Database={database};"
    "Trusted_Connection=yes;"
    )
    conn_str = 'mssql+pyodbc:///?odbc_connect={}'.format(
        urllib.parse.quote_plus(conn_str)
    )

    
    try:
        engine = create_engine(conn_str)
        df = pd.read_sql_table(table, engine)
        return df
    except:
        try:
            df = pd.read_sql_query(query, engine)
            return df
        except Exception as e:
            print("❌ Error:", e)

if __name__ == "__main__":
    df=load_sql_server_data()
df

1433


SQL Server host/IP:  10.21.10.52
Portnumber(default is 1433): 
Database name:  RER_SERVICE
Table name:  sales
write/paste your query 


10.21.10.52,1433


,CreatedAt,AreaSize,SellingPriceAmount,PropertyUse,PropertyType,unittype,ParcelNumber,PlanNumber,DistrictNameAr,CityNameAr,...,SpatialUnitSouthernBorderLength,SpatialUnitEasternBorderLength,SpatialUnitWesternBorderLength,realestate_number,CaseNumber,DeedSerial,DeedNumber,moj_deed_date,new_type,new_use
0,2025-02-25 18:13:17.4132994,400.00,800000.0,Residential,Empty Land,None,13825,2351/7,لبن,الرياض,...,20.19,20.04,19.97,9589793458700000,TR202511593,15002764,498552015311,1/8/1444,أرض,Residential
1,2024-05-21 12:08:00.3156668,260.00,2500000.0,Residential,Residential Building,None,1412/1,1822/س,التعاون,الرياض,...,12.8768,20.2212,20.2081,9591547978400000,TR20242010,1454554,210116024100,22/1/1433,فيلا,Residential
2,2025-03-24 12:22:23.2423048,4994.38,7341739.0,Commercial,Empty Land,None,1594,3880/1,المصفاة,الرياض,...,50.03,100.15,100.38,9615294823200000,TR202516905,10611834,399025000196,4/4/1442,أرض,Commercial
3,2025-01-08 08:43:23.7086187,164.46,600000.0,Mixed Use,Apartment,Apartment,3170,2509,طويق,الرياض,...,30,30.0001,30.0001,9616602812800001,TR20251266,15826616,362001000391,17/3/1445,شقة,Residential
4,2025-04-14 10:41:03.3531231,179.25,650000.0,Mixed Use,Apartment,Apartment,3170,2509,طويق,الرياض,...,30,30.0001,30.0001,9616602812800005,TR202519108,15831443,362090000356,18/3/1445,شقة,Residential
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48445,2024-12-09 05:47:27.4744063,194.68,640625.0,Residential,Apartment,Apartment,85+87+89,ش د 1236,الشعلة,الدمام,...,60.9199,33.5,33.5001,7664109935100013,TR202420703,14829512,330143001843,19/6/1444,شقة,Residential
48446,2024-11-18 16:09:49.2751613,300.00,480000.0,Residential,Empty Land,None,3014,3930,السحاب,الرياض,...,25,12,12,6638339074000000,TR202417298,15772524,399022004835,2/3/1445,أرض,Residential
48447,2024-12-25 11:16:19.0623530,126.73,750000.0,Mixed Use,Residential Building,Apartment,702,2934,النهضة,الرياض,...,29.5288,50.0705,49.99,1920197994700014,TR202423502,None,None,None,شقة,Residential
48448,2024-09-14 16:13:02.0408646,200.34,590000.0,Residential,Empty Land,None,814/3,3537,الخير,الرياض,...,12.3276,16.2517,16.2478,1164237415700000,TR20249598,None,None,None,أرض,Residential


In [212]:
from sqlalchemy import create_engine
from urllib.parse import quote_plus
import pandas as pd
import getpass

def get_postgres_data():
    # Prompt for credentials and connection details
    username = getpass.getpass("Enter PostgreSQL username: ")
    password = getpass.getpass("Enter PostgreSQL password: ")
    host = input("Enter PostgreSQL host (e.g., 127.0.0.1): ")
    port = input("Enter PostgreSQL port (default 5432): ") or "5432"
    database = input("Enter PostgreSQL database name: ")

    # Escape password for URL safety
    encoded_password = quote_plus(password)

    # Build the connection string
    connection_str = f"postgresql+psycopg2://{username}:{encoded_password}@{host}:{port}/{database}"
    engine = create_engine(connection_str)
    try:
        table = input("Enter SQL table: ")
        df = pd.read_sql_table(table, engine)
        print("\n✅ Query Result:")
        return df
    except:
        try:
            query = input("Enter SQL query: ")
            df = pd.read_sql_query(query, engine)
            return df
        except Exception as e:
            print(f"❌ Error: {e}")
df=get_postgres_data()

Enter PostgreSQL username:  ········
Enter PostgreSQL password:  ········
Enter PostgreSQL host (e.g., 127.0.0.1):  10.21.10.32
Enter PostgreSQL port (default 5432):  5432
Enter PostgreSQL database name:  sde
Enter SQL table:  rer_powerbi



✅ Query Result:
   ID_P                                    Id  RealEstateNumber  \
0     3  05a90000-0a64-0a58-539e-08dc94459ae1  8726743626500001   
1     4  06060000-0a64-0a58-8e70-08dc9445d388  2043440074400000   
2     5  06060000-0a64-0a58-6d7b-08dc9445d581  6825597522400000   
3     6  04210000-0a64-0a58-b0f8-08dc9445f07d  5685890444900004   
4     7  06060000-0a64-0a58-1b79-08dc94469224  9262331369800000   

  RealEstateIdentity          RegistrarName          RegistrarNameAr Version  \
0                  0  Abdulsalam Alshamrani  عبدالسلام محمد الشمراني       1   
1                  0        Nawaf Almotairi        نواف صالح المطيري       1   
2                  0         Halaa Alotaibi   هلاء عايض عواض العتيبي       1   
3               None         Halaa Alotaibi   هلاء عايض عواض العتيبي       1   
4               None         Sarah Almutlaq         ساره احمد المطلق       1   

  FirstRegistrationDate PlanNumber BlockNumber  ... IsPublished  \
0  6/24/2024 3:03:05 PM       30

AttributeError: 'NoneType' object has no attribute 'head'

In [25]:
from data_loader import load_data

# SQL ServerEEE
df_sql = load_data('huge_messy_dataset.csv',source_type="csv")

2025-06-26 16:28:56,532 - INFO -  Loading data from source: huge_messy_dataset.csv


csv
csv


2025-06-26 16:28:57,600 - INFO - Loaded data successfully. Shape: (1010000, 9)


In [18]:
import pandas as pd
from data_quality_checker import DataQualityChecker
dq = DataQualityChecker(df, date_column='CreatedAt')
report = dq.run_all_checks()
import os

output_folder = "dq_results"
import os
import pandas as pd
os.makedirs(output_folder, exist_ok=True)  # Create folder if it doesn't exist

# Save all tabular reports
for key, value in report.items():
    if key in ["null_rows", "outlier_rows", "placeholder_rows"]:
        continue  # we'll handle these separately below

    if not isinstance(value, pd.DataFrame):
        try:
            df = pd.DataFrame(value)
        except Exception as e:
            print(f"Skipping {key}, can't convert to DataFrame: {e}")
            continue
    else:
        df = value

    if not df.empty:
        filename = os.path.join(output_folder, f"{key}.csv")
        df.to_csv(filename, index=False)
        print(f"Saved {filename} with {len(df)} records.")
    else:
        print(f"{key} is empty, skipping save.")

# Flatten anomaly dictionaries
flattened_rows = []

def flatten_anomalies(anomaly_dict, anomaly_type):
    combined = []
    for col, df in anomaly_dict.items():
        if not df.empty:
            temp_df = df.copy()
            temp_df["column"] = col
            temp_df["anomaly_type"] = anomaly_type
            combined.append(temp_df)
    return combined

flattened_rows.extend(flatten_anomalies(report.get("null_rows", {}), "null_row"))
flattened_rows.extend(flatten_anomalies(report.get("outlier_rows", {}), "outlier_row"))
flattened_rows.extend(flatten_anomalies(report.get("placeholder_rows", {}), "placeholder_row"))

if flattened_rows:
    combined_df = pd.concat(flattened_rows, ignore_index=True)
    print(combined_df.columns.values)
    combined_path = os.path.join(output_folder, "combined_anomalies.csv")
    combined_df.to_csv(combined_path, index=False)
    print(f"Saved combined anomalies to {combined_path} with {len(combined_df)} records.")
else:
    print("No anomalies to flatten and save.")



Saved dq_results\column_summary.csv with 22 records.
Saved dq_results\nulls.csv with 22 records.
Saved dq_results\outliers.csv with 2 records.
Saved dq_results\duplicates.csv with 1 records.
mixed_types is empty, skipping save.
Saved dq_results\outliers_by_date.csv with 978 records.
Saved dq_results\placeholder_counts.csv with 20 records.
Saved dq_results\placeholder_counts_by_date.csv with 71 records.
Saved dq_results\nulls_by_date.csv with 1882 records.
Saved dq_results\empty_strings_by_date.csv with 26 records.
['CreatedAt' 'AreaSize' 'SellingPriceAmount' 'PropertyUse' 'PropertyType'
 'unittype' 'ParcelNumber' 'PlanNumber' 'DistrictNameAr' 'CityNameAr'
 'RegionNameAr' 'SpatialUnitNorthernBorderLength'
 'SpatialUnitSouthernBorderLength' 'SpatialUnitEasternBorderLength'
 'SpatialUnitWesternBorderLength' 'realestate_number' 'CaseNumber'
 'DeedSerial' 'DeedNumber' 'moj_deed_date' 'new_type' 'new_use' 'column'
 'anomaly_type']
Saved combined anomalies to dq_results\combined_anomalies.csv

In [17]:
    print(combined_df.columns.values)

['ID' 'Name' 'Age' 'Gender' 'Salary' 'JoinDate' 'Department' 'Active'
 'Notes' 'column' 'anomaly_type']


In [1]:
import dash
from dash import html, dcc, dash_table, Input, Output, State
import pandas as pd
import plotly.graph_objects as go
import flask
import threading
import webbrowser
from functools import reduce

# ---------- Helper / small utils ----------
def safe_rename_key_column(df):
    for col in df.columns:
        if col.lower() in ['column', 'column_name', 'key']:
            df.rename(columns={col: 'key'}, inplace=True)
    return df

def get_status_icon(p):
    if p == 0:
        return "✓"
    elif p < 10:
        return "!"
    else:
        return "✖"

def get_unicode_icon(p):
    if p == 0:
        return "\u2705"
    elif p < 10:
        return "\u26A0"
    else:
        return "\u274C"

# ---------- Flask & Dash ----------
server = flask.Flask(__name__)
app = dash.Dash(__name__, server=server)

# ---------- Load data (keep your paths) ----------
null_count = pd.read_csv('dq_results/nulls.csv')
outliers_count = pd.read_csv('dq_results/outliers.csv')
placeholder_count = pd.read_csv('dq_results/placeholder_counts.csv')

null_count = safe_rename_key_column(null_count)
outliers_count = safe_rename_key_column(outliers_count)
placeholder_count = safe_rename_key_column(placeholder_count)

# merge summary
dfs = [null_count, outliers_count, placeholder_count]
summary_data = reduce(lambda left, right: pd.merge(left, right, on='key', how='outer'), dfs)
summary_data.fillna(0, inplace=True)

# compute status and count bar
# assume your columns use names like null_count, outlier_count, placeholder_count etc.
pct_cols = [c for c in summary_data.columns if '_percentage' in c]
cnt_cols = [c for c in summary_data.columns if '_count' in c]

# if percentage columns exist, compute a max-based status
if pct_cols:
    summary_data["Status"] = summary_data[pct_cols].max(axis=1).apply(get_status_icon)
else:
    summary_data["Status"] = summary_data[cnt_cols].apply(lambda r: get_status_icon(0), axis=1)

summary_data["Count Bar"] = summary_data[[c for c in cnt_cols if c in summary_data.columns]].sum(axis=1)

# time series / flagged records
outlier_data = pd.read_csv('dq_results/outliers_by_date.csv')
placeholder_data = pd.read_csv('dq_results/placeholder_counts_by_date.csv')
null_data = pd.read_csv('dq_results/nulls_by_date.csv')
flagged_records = pd.read_excel('dq_results/combined_anomalies.xlsx')

# ---------- Chart helper functions ----------
def all_line(df, title):
    """
    Aggregate across all columns in df and compute:
      - total anomaly counts per month (sum of *_count columns)
      - weighted percentage = total_anomaly_count / total_records (if total_records columns exist)
    """
    if df.empty:
        return go.Figure()

    df = df.copy()
    # normalize date column name
    if 'date_only' not in df.columns and 'date' in df.columns:
        df['date_only'] = df['date']
    df['date_only'] = pd.to_datetime(df['date_only'])
    df['month'] = df['date_only'].dt.to_period('M').dt.to_timestamp()

    numeric_cols = df.select_dtypes(include='number').columns.tolist()

    if not numeric_cols:
        return go.Figure()

    df_agg = df.groupby('month')[numeric_cols].sum().reset_index()

    # counts: any _count column
    count_cols = [c for c in df_agg.columns if '_count' in c and 'total' not in c]
    total_count = df_agg[count_cols].sum(axis=1) if count_cols else pd.Series([0]*len(df_agg))

    # try to find total-record columns (like total_count, total, total_records, etc)
    total_cols = [c for c in df_agg.columns if 'total' in c]
    if total_cols:
        total_records = df_agg[total_cols].sum(axis=1).replace(0, pd.NA)
        perc_col = (total_count / total_records) * 100
        perc_col = perc_col.fillna(0)
    else:
        # fallback: try to derive totals from existing column-specific counts and percentages
        # If any *_percentage columns exist, attempt to compute implied totals where possible.
        perc_cols = [c for c in df_agg.columns if '_percentage' in c]
        if perc_cols and count_cols and len(perc_cols) == len(count_cols):
            # naive attempt: for each paired count/percentage, estimate total = count / (pct/100)
            # then sum totals
            est_totals = []
            for cnt_c, pct_c in zip(count_cols, perc_cols):
                with pd.option_context('mode.use_inf_as_na', True):
                    est_tot = (df_agg[cnt_c] / (df_agg[pct_c] / 100)).replace([pd.NA, float('inf')], 0)
                est_totals.append(est_tot.fillna(0))
            if est_totals:
                total_records = pd.concat(est_totals, axis=1).sum(axis=1).replace(0, pd.NA)
                perc_col = (total_count / total_records) * 100
                perc_col = perc_col.fillna(0)
            else:
                perc_col = pd.Series([0]*len(df_agg))
        else:
            perc_col = pd.Series([0]*len(df_agg))

    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=df_agg['month'],
        y=total_count,
        mode='lines',
        name='Count',
        yaxis='y1',
        fill='tozeroy',
        line_shape='spline'
    ))
    fig.add_trace(go.Scatter(
        x=df_agg['month'],
        y=perc_col,
        mode='lines',
        name='%',
        yaxis='y2',
        fill='tozeroy',
        line_shape='spline'
    ))

    fig.update_layout(
        title=title,
        plot_bgcolor='rgba(0,0,0,0)',
        paper_bgcolor='rgba(0,0,0,0)',
        font={"family": "Calibri", "color": "#333"},
        legend={"orientation": "h", "x": 0.5, "xanchor": "center"},
        xaxis=dict(title='Month', showgrid=False),
        yaxis=dict(title='Count', showgrid=False),
        yaxis2=dict(title='%', overlaying='y', side='right', showgrid=False)
    )
    return fig

def column_area_chart(df, column, title):
    """
    Build series for a specific column (df filtered to rows where df['column'] == column).
    Weighted percentage computed per month using a total column if available.
    """
    if df.empty:
        return go.Figure()

    df_filtered = df[df['column'] == column].copy()
    if df_filtered.empty:
        return go.Figure()

    if 'date_only' not in df_filtered.columns and 'date' in df_filtered.columns:
        df_filtered['date_only'] = df_filtered['date']
    df_filtered['date_only'] = pd.to_datetime(df_filtered['date_only'])
    df_filtered['month'] = df_filtered['date_only'].dt.to_period('M').dt.to_timestamp()

    # find the first count and percentage columns if present
    count_cols = [c for c in df_filtered.columns if '_count' in c and c != 'total_count']
    perc_cols = [c for c in df_filtered.columns if '_percentage' in c]
    total_cols = [c for c in df_filtered.columns if 'total' in c]

    if count_cols:
        count_col = count_cols[0]
    else:
        return go.Figure()

    # aggregate counts
    agg = df_filtered.groupby('month').agg({count_col: 'sum'}).reset_index()

    # percentage: try to compute weighted percentage via total records if present
    if total_cols:
        total_col = total_cols[0]
        totals = df_filtered.groupby('month').agg({total_col: 'sum'}).reset_index()
        agg = agg.merge(totals, on='month', how='left')
        agg['%'] = (agg[count_col] / agg[total_col]) * 100
        agg['%'] = agg['%'].fillna(0)
    elif perc_cols:
        # fallback: average the percentage column (not ideal but better than nothing)
        perc_col = perc_cols[0]
        perc_agg = df_filtered.groupby('month').agg({perc_col: 'mean'}).reset_index()
        agg = agg.merge(perc_agg, on='month', how='left')
        agg['%'] = agg[perc_col].fillna(0)
    else:
        agg['%'] = 0

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=agg['month'], y=agg[count_col], mode='lines', name='Count', yaxis='y1', fill='tozeroy', line_shape='spline'))
    fig.add_trace(go.Scatter(x=agg['month'], y=agg['%'], mode='lines', name='%', yaxis='y2', fill='tozeroy', line_shape='spline'))

    fig.update_layout(
        title=title,
        plot_bgcolor='rgba(0,0,0,0)',
        paper_bgcolor='rgba(0,0,0,0)',
        font={"family": "Calibri", "color": "#333"},
        legend={"orientation": "h", "x": 0.5, "xanchor": "center"},
        xaxis=dict(title='Month', showgrid=False),
        yaxis=dict(title='Count', showgrid=False),
        yaxis2=dict(title='%', overlaying='y', side='right', showgrid=False)
    )
    return fig

def build_cards(source):
    """
    source can be either the full summary DataFrame or a single-row Series (selected column).
    Returns a list of card Divs for null/outlier/placeholder
    """
    # decide if source is a DataFrame or Series
    if isinstance(source, pd.DataFrame):
        # totals across dataset
        null_total = int(source.get('null_count', 0).sum()) if 'null_count' in source else 0
        outlier_total = int(source.get('outlier_count', 0).sum()) if 'outlier_count' in source else 0
        placeholder_total = int(source.get('placeholder_count', 0).sum()) if 'placeholder_count' in source else 0

        null_pct = 0.0
        outlier_pct = 0.0
        placeholder_pct = 0.0
    else:
        # assume Series
        null_total = int(source.get('null_count', 0))
        outlier_total = int(source.get('outlier_count', 0))
        placeholder_total = int(source.get('placeholder_count', 0))

        null_pct = float(source.get('null_percentage', 0))
        outlier_pct = float(source.get('outlier_percentage', 0))
        placeholder_pct = float(source.get('placeholder_percentage', 0))

    cards = [
        html.Div([
            html.H4("Nulls", style={"marginBottom": "5px"}),
            html.Div([
                html.Div(f"{null_total:,}", style={"fontWeight": "bold"}),
                html.Div(f"{null_pct:.2f}% " if null_pct else "(total)", style={"fontSize": "14px"})
            ])
        ], style={"backgroundColor": "#eaeaea", "padding": "10px", "borderRadius": "10px", "width": "20%", "textAlign": "center"}),
        html.Div([
            html.H4("Outliers", style={"marginBottom": "5px"}),
            html.Div([
                html.Div(f"{outlier_total:,}", style={"fontWeight": "bold"}),
                html.Div(f"{outlier_pct:.2f}% " if outlier_pct else "(total)", style={"fontSize": "14px"})
            ])
        ], style={"backgroundColor": "#eaeaea", "padding": "10px", "borderRadius": "10px", "width": "20%", "textAlign": "center"}),
        html.Div([
            html.H4("Placeholders", style={"marginBottom": "5px"}),
            html.Div([
                html.Div(f"{placeholder_total:,}", style={"fontWeight": "bold"}),
                html.Div(f"{placeholder_pct:.2f}% " if placeholder_pct else "(total)", style={"fontSize": "14px"})
            ])
        ], style={"backgroundColor": "#eaeaea", "padding": "10px", "borderRadius": "10px", "width": "20%", "textAlign": "center"})
    ]
    return cards

# ---------- Layout (single, cleaned) ----------
app.layout = html.Div(style={"backgroundColor": "white", "padding": "20px", "fontFamily": "Calibri", "color": "#333"}, children=[
    html.H2("\U0001F4CA Data Quality Profiler", style={"textAlign": "center"}),

    html.Div(id="summary-cards", style={"display": "flex", "gap": "15px", "justifyContent": "center", "marginBottom": "20px"}),

    html.Button("Reset Filters", id="reset-button", n_clicks=0, style={"marginBottom": "10px"}),

    dash_table.DataTable(
        id="summary-table",
        data=summary_data.to_dict("records"),
        columns=[
            {"name": "key", "id": "key", "type": "text"},
            {"name": "null_count", "id": "null_count", "type": "numeric"},
            {"name": "outlier_count", "id": "outlier_count", "type": "numeric"},
            {"name": "placeholder_count", "id": "placeholder_count", "type": "numeric"},
            {"name": "null_percentage", "id": "null_percentage", "type": "numeric", "format": {"specifier": ".2f"}},
            {"name": "outlier_percentage", "id": "outlier_percentage", "type": "numeric", "format": {"specifier": ".2f"}},
            {"name": "placeholder_percentage", "id": "placeholder_percentage", "type": "numeric", "format": {"specifier": ".2f"}},
            {"name": "Count Bar", "id": "Count Bar", "type": "numeric"},
            {"name": "Status", "id": "Status", "type": "text"}
        ],
        sort_action="native",
        filter_action="native",
        row_selectable='single',
        page_size=20,
        style_table={"marginBottom": "30px", "overflowX": "auto"},
        style_cell={
            "padding": "6px",
            "fontFamily": "Calibri",
            "color": "#333",
            "textAlign": "center",
            "minWidth": "100px",
            "maxWidth": "200px",
            "whiteSpace": "normal"
        },
        style_header={
            "backgroundColor": "#f2f2f2",
            "fontWeight": "bold",
            "border": "1px solid #ccc"
        },
        style_data_conditional=[
            {"if": {"row_index": "odd"}, "backgroundColor": "#fafafa"},
            {
                "if": {"filter_query": '{Status} = "✓"', "column_id": "Status"},
                "color": "green", "fontWeight": "bold"
            },
            {
                "if": {"filter_query": '{Status} = "!"', "column_id": "Status"},
                "color": "orange", "fontWeight": "bold"
            },
            {
                "if": {"filter_query": '{Status} = "✖"', "column_id": "Status"},
                "color": "red", "fontWeight": "bold"
            }
        ],
        style_as_list_view=True
    ),

    html.Div([
        dcc.Graph(id="outlier-chart", style={"flex": 1}),
        dcc.Graph(id="null-chart", style={"flex": 1}),
        dcc.Graph(id="placeholder-chart", style={"flex": 1})
    ], style={"display": "flex", "gap": "10px"}),

    html.H3("Flagged Records", style={"marginTop": "30px"}),

    dash_table.DataTable(
        id="flagged-table",
        data=flagged_records.to_dict("records"),
        columns=[{"name": i, "id": i} for i in flagged_records.columns],
        page_size=5,
        style_table={"overflowX": "auto"},
        style_cell={
            "textAlign": "left",
            "padding": "5px",
            "fontFamily": "Calibri",
            "color": "#333"
        },
        style_header={"backgroundColor": "#f2f2f2", "fontWeight": "bold"}
    ),

    html.Div("Created by RMP", style={"textAlign": "right", "marginTop": "20px", "fontStyle": "italic"})
])

# ---------- Callback ----------
@app.callback(
    Output("summary-cards", "children"),
    Output("outlier-chart", "figure"),
    Output("null-chart", "figure"),
    Output("placeholder-chart", "figure"),
    Output("summary-table", "selected_rows"),
    Input("summary-table", "selected_rows"),
    Input("reset-button", "n_clicks"),
    prevent_initial_call=False
)
def update_dashboard(selected_rows, reset_clicks):
    """
    Safe update that handles:
      - no selection (show aggregated views)
      - reset button (clears selection)
      - invalid selection indexes
      - selected column not found
    """

    ctx = dash.callback_context
    # If reset button was clicked, clear selection
    if ctx.triggered and ctx.triggered[0]['prop_id'].startswith('reset-button'):
        return build_cards(summary_data), all_line(outlier_data, "Outliers Over Time"), all_line(null_data, "Nulls Over Time"), all_line(placeholder_data, "Placeholders Over Time"), []

    # No selection → aggregated view

    
    if not selected_rows:
        return build_cards(summary_data), all_line(outlier_data, "Outliers Over Time"), all_line(null_data, "Nulls Over Time"), all_line(placeholder_data, "Placeholders Over Time"), []

    # Try to get the first selected row index safely
    try:
        sel_idx = int(selected_rows[0])
        # ensure sel_idx within bounds
        if sel_idx < 0 or sel_idx >= len(summary_data):
            raise IndexError("selected index out of range")
        column_name = summary_data.iloc[sel_idx]['key']
    except Exception:
        # fallback to aggregated view
        return build_cards(summary_data), all_line(outlier_data, "Outliers Over Time"), all_line(null_data, "Nulls Over Time"), all_line(placeholder_data, "Placeholders Over Time"), []

    # ensure selected column exists in summary_data
    col_series = summary_data[summary_data['key'] == column_name]
    if col_series.empty:
        return build_cards(summary_data), all_line(outlier_data, "Outliers Over Time"), all_line(null_data, "Nulls Over Time"), all_line(placeholder_data, "Placeholders Over Time"), []

    col_counts = col_series.iloc[0]

    # build area charts for that specific column from each timeseries df
    outlier_fig = column_area_chart(outlier_data, column_name, f"{column_name} - Outliers Over Time")
    null_fig = column_area_chart(null_data, column_name, f"{column_name} - Nulls Over Time")
    placeholder_fig = column_area_chart(placeholder_data, column_name, f"{column_name} - Placeholders Over Time")

    return build_cards(col_counts), outlier_fig, null_fig, placeholder_fig, selected_rows

# ---------- Auto-open browser and run ----------
def open_browser():
    webbrowser.open_new("http://127.0.0.1:8050")

if __name__ == "__main__":
    threading.Timer(1.5, open_browser).start()
    app.run(debug=True)


In [21]:
import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta

# Set seed for reproducibility
np.random.seed(42)
random.seed(42)

# Record count
n = 1_000_000

# Generate core data
df = pd.DataFrame({
    "ID": np.arange(1, n + 1),
    "Name": np.random.choice(['Alice', 'Bob', 'Charlie', 'N/A', '', 'Unknown', 'Z@ck', 'A1'], size=n),
    "Age": np.random.choice(list(range(18, 70)) + [None, 'Unknown', 'Thirty'], size=n),
    "Gender": np.random.choice(['Male', 'Female', 'Other', 'N/A', '', None], size=n),
    "Salary": np.append(
        np.random.normal(50000, 15000, size=int(n * 0.98)),
        np.random.normal(500000, 100000, size=int(n * 0.02))  # outliers
    ),
    "Department": np.random.choice(['HR', 'Finance', 'IT', 'Unknown', 'Operations', 'other', None], size=n),
    "Active": np.random.choice(['Yes', 'No', '1', '0', 'True', 'False', None, 'N/A'], size=n),
})

# Notes column with mixed types
df["Notes"] = np.random.choice([
    "Good", "Check", "Error", None, "", 123, 45.6, True, pd.NaT, "!!!"
], size=n)

# Duplicate ~1% of rows to simulate duplicates
duplicates = df.sample(frac=0.01, replace=True)
df = pd.concat([df, duplicates], ignore_index=True)
df = df.sample(frac=1).reset_index(drop=True)

# ----------- Generate realistic + messy JoinDate values ------------
def random_date(start, end):
    delta = end - start
    return start + timedelta(days=random.randint(0, delta.days))

# Generate clean formatted dates in different styles over 5 years
clean_formats = ['%Y-%m-%d', '%d/%m/%Y', '%Y/%m/%d']
clean_dates = [
    random_date(datetime(2020, 1, 1), datetime(2024, 12, 31)).strftime(fmt)
    for fmt in clean_formats
    for _ in range((len(df) // 10) // len(clean_formats))
]

# Add messy entries
messy_entries = random.choices([
    'N/A', '', None, 'Not Available', '---', '2020-00-00', '2021-13-01'
], k=len(df) - len(clean_dates))

# Combine and shuffle
all_dates = clean_dates + messy_entries
random.shuffle(all_dates)

# Assign to DataFrame
df['JoinDate'] = all_dates[:len(df)]  # Ensure exact match

# ---------------- Save to CSV ----------------
df.to_csv("huge_messy_dataset.csv", index=False)
print(f"Saved {len(df)} rows to huge_messy_dataset.csv ✅")


Saved 1010000 rows to huge_messy_dataset.csv ✅


In [1]:
import dash
from dash import html, dcc, dash_table, Input, Output, State
import pandas as pd
import plotly.graph_objects as go
import flask
import threading
import webbrowser
from datetime import datetime
from functools import reduce

def generate_table_data(df):
    table_data = df.copy()
    table_data['Count Bar'] = table_data['Count']
    
    def get_icon(percent):
        if percent < 10:
            return "✓"
        elif percent < 30:
            return "!"
        else:
            return "✖"
    
    table_data['Status Icon'] = table_data['Percentage'].apply(get_icon)
    return table_data

# ---------- Flask Server ----------
server = flask.Flask(__name__)
app = dash.Dash(__name__, server=server)

# ---------- Sample Data ----------
null_count = pd.read_csv('dq_results/nulls.csv')
outliers_count = pd.read_csv('dq_results/outliers.csv')
placeholder_count = pd.read_csv('dq_results/placeholder_counts.csv')

# Ensure consistent column name for merge key
def rename_key_column(df):
    for col in df.columns:
        if col.lower() in ['column', 'column_name', 'key']:
            df.rename(columns={col: 'key'}, inplace=True)
    return df

null_count = rename_key_column(null_count)
outliers_count = rename_key_column(outliers_count)
placeholder_count = rename_key_column(placeholder_count)

dfs = [null_count, outliers_count, placeholder_count]

# Full outer join all dfs on 'key'
summary_data = reduce(lambda left, right: pd.merge(left, right, on='key', how='outer'), dfs)

# Replace NaN with 0
summary_data.fillna(0, inplace=True)

def get_icon(p):
    if p == 0:
        return "✓"
    elif p < 10:
        return "!"
    else:
        return "✖"

summary_data["Status"] = summary_data[['null_percentage', 'outlier_percentage', 'placeholder_percentage']].max(axis=1).apply(get_icon)
summary_data["Count Bar"] = summary_data[['null_count', 'outlier_count', 'placeholder_count']].sum(axis=1)

outlier_data = pd.read_csv('dq_results/outliers_by_date.csv')
placeholder_data = pd.read_csv('dq_results/placeholder_counts_by_date.csv')
null_data = pd.read_csv('dq_results/nulls_by_date.csv')
flagged_records = pd.read_excel('dq_results/combined_anomalies.xlsx')

# ---------- Utilities ----------
def get_icon(p):
    if p == 0:
        return "\u2705"
    elif p < 10:
        return "\u26A0"
    else:
        return "\u274C"

# ---------- Layout ----------
app.layout = html.Div(style={
    "backgroundColor": "white",
    "padding": "20px",
    "fontFamily": "Calibri",
    "color": "#333",
    "fontSize": "18px"  # Base font size increased
}, children=[
    html.H2("\U0001F4CA Data Quality Profiler", style={
        "textAlign": "center",
        "fontSize": "32px"
    }),

    html.Div(id="summary-cards", style={
        "display": "flex",
        "gap": "15px",
        "justifyContent": "center",
        "marginBottom": "20px"
    }),

    html.Button("Reset Filters", id="reset-button", n_clicks=0, style={
        "marginBottom": "10px",
        "fontSize": "16px",
        "padding": "8px 15px"
    }),

    # Main Summary Table
    dash_table.DataTable(
        id="summary-table",
        data=summary_data.to_dict("records"),
        columns=[
            {"name": "key", "id": "key", "type": "text"},
            {"name": "null_count", "id": "null_count", "type": "numeric"},
            {"name": "outlier_count", "id": "outlier_count", "type": "numeric"},
            {"name": "placeholder_count", "id": "placeholder_count", "type": "numeric"},
            {"name": "null_percentage", "id": "null_percentage", "type": "numeric", "format": {"specifier": ".2f"}},
            {"name": "outlier_percentage", "id": "outlier_percentage", "type": "numeric", "format": {"specifier": ".2f"}},
            {"name": "placeholder_percentage", "id": "placeholder_percentage", "type": "numeric", "format": {"specifier": ".2f"}},
            {"name": "Count Bar", "id": "Count Bar", "type": "numeric"},
            {"name": "Status", "id": "Status", "type": "text"}
        ],
        sort_action="native",
        filter_action="native",
        row_selectable='single',
        page_size=20,
        style_table={"marginBottom": "30px", "overflowX": "auto"},
        style_cell={
            "padding": "10px",
            "fontFamily": "Calibri",
            "color": "#333",
            "textAlign": "center",
            "fontSize": "16px",  # Increased font size
            "minWidth": "100px",
            "maxWidth": "200px",
            "whiteSpace": "normal"
        },
        style_header={
            "backgroundColor": "#f2f2f2",
            "fontWeight": "bold",
            "border": "1px solid #ccc",
            "fontSize": "17px"  # Increased header font size
        },
        style_data_conditional=[
            {"if": {"row_index": "odd"}, "backgroundColor": "#fafafa"},

            # Status icon colors
            {
                "if": {"filter_query": '{Status} = "✓"', "column_id": "Status"},
                "color": "green", "fontWeight": "bold"
            },
            {
                "if": {"filter_query": '{Status} = "!"', "column_id": "Status"},
                "color": "orange", "fontWeight": "bold"
            },
            {
                "if": {"filter_query": '{Status} = "✖"', "column_id": "Status"},
                "color": "red", "fontWeight": "bold"
            },

            # Data bar for Count Bar column
            {
                "if": {"column_id": "Count Bar"},
                "background": (
                    "linear-gradient(90deg, #007bff 0%, "
                    "#007bff calc(min(100%, ({Count Bar} / 1000) * 100%)), "
                    "transparent calc(min(100%, ({Count Bar} / 1000) * 100%)))"
                ),
                "color": "#000"
            }
        ],
        style_as_list_view=True
    ),

    # Charts in one row
    html.Div([
        dcc.Graph(id="outlier-chart", style={"flex": 1}),
        dcc.Graph(id="null-chart", style={"flex": 1}),
        dcc.Graph(id="placeholder-chart", style={"flex": 1})
    ], style={"display": "flex", "gap": "10px"}),

    # Flagged Records section
    html.H3("Flagged Records", style={
        "marginTop": "30px",
        "fontSize": "26px"
    }),

    dash_table.DataTable(
        data=flagged_records.to_dict("records"),
        columns=[{"name": i, "id": i} for i in flagged_records.columns],
        page_size=5,
        style_table={"overflowX": "auto"},
        style_cell={
            "textAlign": "left",
            "padding": "10px",
            "fontFamily": "Calibri",
            "color": "#333",
            "fontSize": "16px"
        },
        style_header={
            "backgroundColor": "#f2f2f2",
            "fontWeight": "bold",
            "fontSize": "17px"
        }
    ),

    html.Div("Created by RMP", style={
        "textAlign": "right",
        "marginTop": "20px",
        "fontStyle": "italic",
        "fontSize": "14px"
    })
])

# ---------- Callbacks ----------
@app.callback(
    [Output("summary-cards", "children"),
     Output("outlier-chart", "figure"),
     Output("null-chart", "figure"),
     Output("placeholder-chart", "figure"),
     Output("summary-table", "selected_rows")],
    [Input("summary-table", "selected_rows"),
     Input("reset-button", "n_clicks")],
    [State("summary-table", "selected_rows")]
)
def update_dashboard(selected_rows, reset_clicks, state_selected_rows):
    if dash.callback_context.triggered_id == "reset-button":
        selected_rows = []

    column = summary_data.iloc[selected_rows[0]]['key'] if selected_rows else None

    if column is None:
        cards = [
            html.Div([
                html.H4(metric, style={"marginBottom": "5px", "fontSize": "20px"}),
                html.Div([
                    html.Div(f"{summary_data[c + '_count'].sum():,}", style={"fontWeight": "bold", "fontSize": "22px"}),
                    html.Div("(total)", style={"fontSize": "16px"})
                ])
            ], style={"backgroundColor": "#eaeaea", "padding": "10px", "borderRadius": "10px", "width": "18%"})
            for metric, c in zip(["nulls", "outlier", "placeholder"], ["null", "outlier", "placeholder"])
        ]

        def all_line(df, title):
            df['date_only'] = pd.to_datetime(df['date_only'])
            df['month'] = df['date_only'].dt.to_period('M').dt.to_timestamp()
            numeric_df = df.drop(columns=['date_only', 'column', 'month'], errors='ignore')
            df_agg = numeric_df.groupby(df['month']).sum().reset_index()
            count_col = df_agg.filter(like='_count').sum(axis=1)
            percentage_cols = df_agg.filter(like='_percentage')
            if not percentage_cols.empty:
                perc_col = percentage_cols.mean(axis=1)
            else:
                perc_col = pd.Series([0] * len(df_agg))
            fig = go.Figure()
            fig.add_trace(go.Scatter(
                x=df_agg['month'], y=count_col,
                mode='lines', name='Count',
                yaxis='y1', fill='tozeroy', line_shape='spline'))
            fig.add_trace(go.Scatter(
                x=df_agg['month'], y=perc_col,
                mode='lines', name='%',
                yaxis='y2', fill='tozeroy', line_shape='spline'))
            fig.update_layout(
                title=title,
                plot_bgcolor='rgba(0,0,0,0)',
                paper_bgcolor='rgba(0,0,0,0)',
                font={"family": "Calibri", "color": "#333", "size": 16},
                legend={"orientation": "h", "x": 0.5, "xanchor": "center"},
                xaxis=dict(title='Month', showgrid=False, titlefont={"size": 16}, tickfont={"size": 14}),
                yaxis=dict(title='Count', showgrid=False, titlefont={"size": 16}, tickfont={"size": 14}),
                yaxis2=dict(title='%', overlaying='y', side='right', showgrid=False, titlefont={"size": 16}, tickfont={"size": 14})
            )
            return fig

        return cards, all_line(outlier_data, "Outliers Over Time"), all_line(null_data, "Nulls Over Time"), all_line(placeholder_data, "Placeholders Over Time"), []

    # When a column is selected:
    col_counts = summary_data[summary_data['key'] == column].iloc[0]
    cards = []
    for anomaly_type in ['null', 'outlier', 'placeholder']:
        cards.append(html.Div([
            html.H4(anomaly_type.capitalize(), style={"marginBottom": "5px", "fontSize": "20px"}),
            html.Div([
                html.Div(f"{col_counts[anomaly_type + '_count']:,}", style={"fontWeight": "bold", "fontSize": "22px"}),
                html.Div(f"{col_counts[anomaly_type + '_percentage']:.2f}% {get_icon(col_counts[anomaly_type + '_percentage'])}", style={"fontSize": "16px"})
            ])
        ], style={"backgroundColor": "#eaeaea", "padding": "10px", "borderRadius": "10px", "width": "18%"}))

    def make_area_chart(df, col, title):
        df_filtered = df[df['column'] == col].copy()
        if df_filtered.empty:
            return go.Figure()
    
        df_filtered['date_only'] = pd.to_datetime(df_filtered['date_only'])
        df_filtered['month'] = df_filtered['date_only'].dt.to_period('M').dt.to_timestamp()
    
        count_col = [c for c in df_filtered.columns if '_count' in c and c != 'total_count']
        perc_col = [c for c in df_filtered.columns if '_percentage' in c]
    
        if not count_col or not perc_col:
            return go.Figure()
    
        agg_df = df_filtered.groupby('month').agg({
            count_col[0]: 'sum',
            perc_col[0]: 'mean'
        }).reset_index()
    
        fig = go.Figure()
        fig.add_trace(go.Scatter(
            x=agg_df['month'], y=agg_df[count_col[0]],
            mode='lines', name='Count',
            yaxis='y1', fill='tozeroy', line_shape='spline'))
        fig.add_trace(go.Scatter(
            x=agg_df['month'], y=agg_df[perc_col[0]],
            mode='lines', name='%',
            yaxis='y2', fill='tozeroy', line_shape='spline'))
        fig.update_layout(
            title=title,
            plot_bgcolor='rgba(0,0,0,0)',
            paper_bgcolor='rgba(0,0,0,0)',
            font={"family": "Calibri", "color": "#333", "size": 16},
            legend={"orientation": "h", "x": 0.5, "xanchor": "center"},
            xaxis=dict(title='Month', showgrid=False, titlefont={"size": 16}, tickfont={"size": 14}),
            yaxis=dict(title='Count', showgrid=False, titlefont={"size": 16}, tickfont={"size": 14}),
            yaxis2=dict(title='%', overlaying='y', side='right', showgrid=False, titlefont={"size": 16}, tickfont={"size": 14})
        )
        return fig

    # Create figures for selected column
    outlier_fig = make_area_chart(outlier_data, column, "Outliers Over Time")
    null_fig = make_area_chart(null_data, column, "Nulls Over Time")
    placeholder_fig = make_area_chart(placeholder_data, column, "Placeholders Over Time")

    return cards, outlier_fig, null_fig, placeholder_fig, selected_rows

# ---------- Auto Open in Browser ----------
def open_browser():
    webbrowser.open_new("http://127.0.0.1:8050")

if __name__ == "__main__":
    threading.Timer(1.5, open_browser).start()
    app.run(debug=True)
